In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

True

In [ ]:
pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# import wandb # logger
import math
from collections import OrderedDict
import h5py
import os
import pickle
import matplotlib.pyplot as plt

In [ ]:
# 3 login to logger
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.init(
    project="FCSN_ATTENTION_FU_1",
    config={
        "epochs": 20,
        "batch_size": 5,
        "lr": 0.0001,
        "decay_rate": 0.8,
        "fusion_dimention": 1200,
        "dropout": 0.3,
        "Description": "300 fusion dimention, train 234 test 1"
    }
)

In [ ]:
config = {
  "max_segment_num": 20,
  "max_frame_num": 200,
  "similarity_dim": 1000,
  "concept_dim": 300,
  "in_channel": 2048,
  "conv1_channel": 512,
  "conv2_channel": 256,
  "deconv1_channel": 1024,
  "deconv2_channel": 1024,

  "gpu": "0",
  "num_workers": 0,
  "epoch": 20,
  "batch_size": 5,
  "lr": 0.0001,
  "train_videos": [1,2,3],
  "test_video": 4,
  "top_percent": 0.02
}

In [ ]:

# Function to load the pickle file
def load_pickle(filename):
    with open(filename,'rb') as f:
        return pickle.load(f)

In [ ]:
# Custom class to load the dataset
class UTEDataset(Dataset):
    def __init__(self, videos):
        self.dataset = []
        train_videos = videos
        for video_id in train_videos:
            for _, _, files in os.walk(f"/content/drive/MyDrive/Research/data/Oracle_summaries/P0{video_id}"):
                for file in files:
                    self.dataset.append(file[:file.find("_oracle.txt")]+f"_{video_id}") # eg. Food_Hands_1 -> (q1_q2_video_id)
        self.embedding=load_pickle("/content/drive/MyDrive/Research/data/query_dictionary.pkl")

    def list_dataset(self):
        return self.dataset

    def __getitem__(self, index):
        """
        This method will be callded when access indexed.
        """
        video_id = self.dataset[index].split("_")[2] # get video id from Food_Hands_1 1 here is video id
        f = h5py.File(f'/content/drive/MyDrive/Research/data/processed/V{video_id}_resnet_avg.h5', 'r') # loading features
        features=torch.tensor(f["features"][()], dtype=torch.float32)
        seg_len=torch.tensor(f["seg_len"][()], dtype=torch.int32)


        query_1, query_2 = self.dataset[index].split("_")[0:2]

        query_1_GT = torch.zeros(20*200) # GT of shape 20*200 (max_seg_size*max_shot_size)
        query_2_GT = torch.zeros(20*200)

        transfer={"Cupglass":"Glass",
                  "Musicalinstrument":"Instrument",
                  "Petsanimal":"Animal"}

        with open(f"/content/drive/MyDrive/Research/data/Dense_per_shot_tags/P0{video_id}/P0{video_id}.txt", "r") as f:
            lines = f.readlines()
            for index, line in enumerate(lines):
                queries = line.strip().split(',')
                if query_1 in queries:
                    query_1_GT[index] = 1
                if query_2 in queries:
                    query_2_GT[index] = 1
                # NO need of the condition given below 🟥
                # if index == 19:
                #     break

        shot_num = seg_len.sum() # getting total count of shots by summing individual segment length
        mask_GT = torch.zeros(20*200, dtype=torch.bool)
        for i in range(shot_num):
            mask_GT[i] = 1

        if query_1 in transfer:
            query_1=transfer[query_1]
        if query_2 in transfer:
            query_2=transfer[query_2]

        q1_text = query_1
        q2_text = query_2
        # print("Q1: ", query_1)
        # print("Q2: ", query_2)
        query_1 = torch.tensor(self.embedding[query_1], dtype=torch.float32)
        query_2 = torch.tensor(self.embedding[query_2], dtype=torch.float32)

        return features, seg_len, query_1, query_2, query_1_GT, query_2_GT, mask_GT, q1_text, q2_text

    def __len__(self):
        return len(self.dataset)

In [ ]:
dtst = UTEDataset([1,2,3])

In [ ]:
len(dtst[0])

9

In [ ]:
class Attention(torch.nn.Module):
    def __init__(self, Q_dim, K_dim, head_size):
        """
        head_size: it will be 256 but can be changed and its being used to project into same feature dimentions of q, k , and v.
        """
        super().__init__()
        self.query = torch.nn.Linear(Q_dim, head_size, bias=False)
        self.key = torch.nn.Linear(K_dim, head_size, bias=False)
        self.value = torch.nn.Linear(K_dim, head_size, bias=False)

    def forward(self, Q_in, K_in, attention_mask):
        q = self.query(Q_in)
        k = self.key(K_in) # (B,T,C) same for all q, k, v
        v = self.value(K_in)

        print(f"k shape : {k.shape} | q shape: {q.shape}")

        wei = q @ k.transpose(1,2)  * (1 / math.sqrt(q.size(-1)))  # (B, T, 256) @ (B, 256, T)   ---> (B, T, T)
        print("wei shape: ", wei.shape)
        # wei = torch.zeros((T,T))
        wei = wei.masked_fill(attention_mask == 0, float('-inf'))
        # wei = wei.masked_fill(attention_mask[:,:50,:50] == 0, float('-inf'))
        # wei=wei.masked_fill(~attention_mask,-1e10)
        wei = F.softmax(wei, dim=-1)

        out = wei @ v
        return out, wei

In [ ]:
import math
import torch
import torch.nn as nn
# from .attention import Attention


class CHAN(nn.Module):
    def __init__(self,config, n_class=256):
        nn.Module.__init__(self)
        self.config=config

        self.conv1 = nn.Sequential(OrderedDict([
            ('conv1_1', nn.Conv1d(2048, 2048, 3, padding=1)),
            ('bn1_1', nn.BatchNorm1d(2048)),
            ('relu1_1', nn.ReLU(inplace=True)),
            ('conv1_2', nn.Conv1d(2048, 2048, 3, padding=1)),
            ('bn1_2', nn.BatchNorm1d(2048)),
            ('relu1_2', nn.ReLU(inplace=True)),
            ('pool1', nn.MaxPool1d(2, stride=2, ceil_mode=True))
            ])) # 1/2

        self.conv2 = nn.Sequential(OrderedDict([
            ('conv2_1', nn.Conv1d(2048, 2048, 3, padding=1)),
            ('bn2_1', nn.BatchNorm1d(2048)),
            ('relu2_1', nn.ReLU(inplace=True)),
            ('conv2_2', nn.Conv1d(2048, 2048, 3, padding=1)),
            ('bn2_2', nn.BatchNorm1d(2048)),
            ('relu2_2', nn.ReLU(inplace=True)),
            ('pool2', nn.MaxPool1d(2, stride=2, ceil_mode=True))
            ])) # 1/4

        self.conv3 = nn.Sequential(OrderedDict([
            ('conv3_1', nn.Conv1d(2048, 2048, 3, padding=1)),
            ('bn3_1', nn.BatchNorm1d(2048)),
            ('relu3_1', nn.ReLU(inplace=True)),
            ('conv3_2', nn.Conv1d(2048, 2048, 3, padding=1)),
            ('bn3_2', nn.BatchNorm1d(2048)),
            ('relu3_2', nn.ReLU(inplace=True)),
            ('conv3_3', nn.Conv1d(2048, 2048, 3, padding=1)),
            ('bn3_3', nn.BatchNorm1d(2048)),
            ('relu3_3', nn.ReLU(inplace=True)),
            ('pool3', nn.MaxPool1d(2, stride=2, ceil_mode=True))
            ])) # 1/8

        self.conv4 = nn.Sequential(OrderedDict([
            ('conv4_1', nn.Conv1d(2048, 2048, 3)),
            ('bn4_1', nn.BatchNorm1d(2048)),
            ('relu4_1', nn.ReLU(inplace=True)),
            ('conv4_2', nn.Conv1d(2048, 2048, 3)),
            ('bn4_2', nn.BatchNorm1d(2048)),
            ('relu4_2', nn.ReLU(inplace=True)),
            ('conv4_3', nn.Conv1d(2048, 2048, 3)),
            ('bn4_3', nn.BatchNorm1d(2048)),
            ('relu4_3', nn.ReLU(inplace=True)),
            # ('pool4', nn.MaxPool1d(2, stride=2, ceil_mode=True))
            ])) # 1/16

        self.conv5 = nn.Sequential(OrderedDict([
            ('conv5_1', nn.Conv1d(2048, 2048, 3, padding=1)),
            ('bn5_1', nn.BatchNorm1d(2048)),
            ('relu5_1', nn.ReLU(inplace=True)),
            ('conv5_2', nn.Conv1d(2048, 2048, 3, padding=1)),
            ('bn5_2', nn.BatchNorm1d(2048)),
            ('relu5_2', nn.ReLU(inplace=True)),
            ('conv5_3', nn.Conv1d(2048, 2048, 3, padding=1)),
            ('bn5_3', nn.BatchNorm1d(2048)),
            ('relu5_3', nn.ReLU(inplace=True)),
            ('pool5', nn.MaxPool1d(2, stride=2, ceil_mode=True))
            ])) # 1/32

        self.conv6 = nn.Sequential(OrderedDict([
            ('fc6', nn.Conv1d(2048, 4096, 1)),
            ('bn6', nn.BatchNorm1d(4096)),
            ('relu6', nn.ReLU(inplace=True)),
            ('drop6', nn.Dropout(0.3))
            ]))

        self.conv7 = nn.Sequential(OrderedDict([
            ('fc7', nn.Conv1d(4096, 4096, 1)),
            ('bn7', nn.BatchNorm1d(4096)),
            ('relu7', nn.ReLU(inplace=True)),
            ('drop7', nn.Dropout(0.3))
            ]))

        self.conv8 = nn.Sequential(OrderedDict([
            ('fc8', nn.Conv1d(4096, n_class, 1)),
            ('bn8', nn.BatchNorm1d(n_class)),
            ('relu8', nn.ReLU(inplace=True)),
            ]))

        self.conv_pool4 = nn.Conv1d(2048, 1024, 1)
        self.bn_pool4 = nn.BatchNorm1d(1024)

        # self.conv1d_1=nn.Conv1d(self.config["in_channel"],self.config["conv1_channel"],kernel_size=5,stride=1,padding=2)
        # self.max_pooling_1=nn.MaxPool1d(2,stride=2,padding=0)
        # self.conv1d_2=nn.Conv1d(self.config["conv1_channel"],self.config["conv2_channel"],kernel_size=5,stride=1,padding=2)
        # self.max_pooling_2=nn.MaxPool1d(2,stride=2,padding=0)
        # self.self_attention=Attention(self.config["conv2_channel"],self.config["conv2_channel"],self.config["conv2_channel"])
        # self.concept_attention=Attention(self.config["concept_dim"],self.config["conv2_channel"],self.config["conv2_channel"])

        # self.self_attention=Attention(256,256,256)
        # self.concept_attention=Attention(300,256,256)

        self.self_attention=Attention(self.config["conv2_channel"],self.config["conv2_channel"],self.config["conv2_channel"])
        self.query_relevence_attention=Attention(self.config["concept_dim"],self.config["conv2_channel"],self.config["conv2_channel"])
        self.global_attention=Attention(self.config["conv2_channel"],self.config["conv2_channel"],self.config["conv2_channel"])


        self.deconv1 = nn.ConvTranspose1d(768, 1024, 3, padding=1, stride=2, bias=False)
        self.deconv2 = nn.ConvTranspose1d(1024, 1024, 20, stride=10, bias=False)

        self.similarity_linear1=torch.nn.Linear(1024,300,bias=False)
        self.similarity_linear2=torch.nn.Linear(300,300,bias=False)

        # self.MLP=torch.nn.Linear(self.config["similarity_dim"],1)
        self.MLP = torch.nn.Linear(1200, 1)

    # batch tensor: batch_size * max_seg_num * max_seg_length * 2048/4096
    # seg_len list(list(int)) : batch_size * seg_num (num of frame)
    # concept : batch_size * 300
    def forward(self,batch,seg_len,concept1,concept2):
        batch_size=batch.size()[0]
        max_seg_num=batch.size()[1]
        max_seg_length=batch.size()[2]
        B, seg, T, C = batch.shape

        # print("Seg Len: ", seg_len)
        # print("batch_size: ", batch_size)
        # print("Concept Shape: ", concept1.shape)
        # print("Conv1d in shape-> ", batch.view(batch_size*max_seg_num,max_seg_length,-1).transpose(1,2).shape)
        # (batch_size * max_seg_num) * 128 * max_seg_length
        # tmp1=self.conv1d_1(batch.view(batch_size*max_seg_num,max_seg_length,-1).transpose(1,2))

        # # print("Conv1d : ", tmp1.shape)
        # # (batch_size * max_seg_num) * 128 * max_seg_length/2
        # tmp1=self.max_pooling_1(tmp1)

        # # print("Max Pooling: ", tmp1.shape)

        # # (batch_size * max_seg_num) * 256 * max_seg_length/2
        # tmp2=self.conv1d_2(tmp1)

        # # print("Conv2 : ", tmp2.shape)
        # # (batch_size * max_seg_num) * max_seg_length/4 * 256
        # tmp2=self.max_pooling_2(tmp2).transpose(1,2)
        h = batch.view(batch_size*max_seg_num, max_seg_length,-1).transpose(1,2)
        h = self.conv1(h)
        print(f"conv1 {h.shape}")
        h = self.conv2(h)
        print(f"conv2 {h.shape}")
        h = self.conv3(h)
        print(f"conv3 {h.shape}")
        # print("H's Shape: ", h.shape)
        h = self.conv4(h)
        print(f"conv4 {h.shape}")
        pool4 = h

        # print("H's Shape: ", h.shape)

        h = self.conv5(h)
        print(f"conv5 {h.shape}")
        h = self.conv6(h)
        print(f"conv6 {h.shape}")
        h = self.conv7(h)
        print(f"conv7 {h.shape}")
        # print("H's Shape: ", h.shape)
        h = self.conv8(h)
        print(f"conv8 {h.shape}")
        h = h.transpose(1,2)

        # print("H's Shape: ", h.shape) # [20, 1024, 7] --> (B*seg, C, T)

        # print("Max Pool : ", tmp2.shape)
        # batch_size * max_seg_num * max_seg_length/4

        # attention_mask=torch.zeros(batch_size,max_seg_num,int(max_seg_length/20),dtype=torch.bool)
        # for i in range(batch_size):
        #     print("i: ", i)
        #     print("len: ", len(seg_len[i]))
        #     for j in range(len(seg_len[i])):
        #         for k in range(int(math.ceil(seg_len[i][j]/20.0))):
        #             attention_mask[i][j][k]=1

        # tmp2 = tmp2.transpose(1,2)

        # print("Max Pool : ", tmp2.shape)
        # batch_size * max_seg_num * max_seg_length/4
        attention_mask=torch.zeros(batch_size,max_seg_num,int(max_seg_length/20),dtype=torch.bool).to('cuda')
        for i in range(batch_size):
            # print("i: ", i)
            # print("len: ", len(seg_len[i]))
            for j in range(len(seg_len[i])):
                for k in range(int(math.ceil(seg_len[i][j]/20.0))):
                    attention_mask[i][j][k]=1

        # print("Attention mask: ", attention_mask.shape) # 📍

        # (batch_size * max_seg_num) * max_seg_length/4 * max_seg_length/4
        # attention_mask=attention_mask.view(batch_size*max_seg_num,-1).unsqueeze(1)
        attention_mask=attention_mask.view(B*seg,-1).unsqueeze(1) # (B*seg, 1, T/4) --> (200, 1, 50)

        q_mat = h.view(B*seg, int(T//20), -1)
        k_mat = h.view(B*seg, int(T//20), -1)

        #### self-attention
        self_attention_result, self_attention_wei = self.self_attention(q_mat, k_mat, attention_mask)

        #### query_relevence_attention

        # average representation of both query
        avg_query = (concept1 * concept2)/2
        t_q = avg_query.unsqueeze(1).unsqueeze(1).expand(
                    B,seg,int(
                        T/20
                    ),300).contiguous().view(B*seg,int(T/20),300)

        query_relevance_attention_result, query_relevance_attention_wei = self.query_relevence_attention(t_q, k_mat, attention_mask)

        query_relevance_attention_agg_result = query_relevance_attention_result.mean(dim=1) # (B*seg, C) -> (200, 256) which is segment wise query relevence score, 1 vector per segment
        query_relevance_attention_agg_result = query_relevance_attention_agg_result.unsqueeze(0).expand(int(T//20),B*seg,256)
        q_mat = q_mat.contiguous().view(int(T/20), B*seg, 256) # (B*seg, T/20, 256) --> (T/20, B*seg, 256) == (50, 200, 256)

        ####  Global Attention
        attention_mask = attention_mask.view(int(T//20), 1, B*seg) # (B*seg, 1, T/20) --> (T/20, 1, B*seg)
        global_attention_result, global_attention_wei = self.global_attention(q_mat,query_relevance_attention_agg_result, attention_mask) # (T/20, B*seg, 256)
        global_attention_result = global_attention_result.view(B*seg, int(T//20), -1) # (T/20, B*seg, 256) --> (B*seg, T/20, 256)

        ## Concat the feature
        final_concat_result=torch.cat((h,self_attention_result,global_attention_result),dim=-1) # (B*seg, T/4, 768)

        # (batch_size * max_seg_num) * max_seg_length/4 * 1024
        # attention_result=torch.cat((h,self_attention_result,concept1_attention_result,concept2_attention_result),dim=-1)


        # print("Attn concat result: ", attention_result.shape)

        attention_result = final_concat_result.transpose(1,2)
        # print("Attn transpose result: ", attention_result.shape)
        h = self.deconv1(attention_result)
        print(f"deconv1 {h.shape}")
        upscore2 = h
        h = self.conv_pool4(pool4)
        h = self.bn_pool4(h)
        score_pool4 = h
        # print("upscore2: ", upscore2.shape)
        # print("score_pool4: ", score_pool4.shape)

        h = upscore2 + score_pool4

        h = self.deconv2(h)
        print(f"deconv2 {h.shape}")

        # print("final deconv op: ", h.shape)
        h = h.transpose(1,2)
        # print("final deconv op tp: ", h.shape)
        # print("Cat attention_result : ", attention_result.shape) # 📍
        # (batch_size * max_seg_num) * 200 * max_seg_length/2
        # result=self.transpose_conv1d_1(attention_result.transpose(1,2))

        # print("deconv 1 ip: ", attention_result.transpose(1,2).shape)
        # print("deconv 1: ", result.shape)
        # print("deconv 2: ip ", self.transpose_conv1d_2(result).transpose(1,2).shape)

        # batch_size * max_seg_num * max_seg_length * 400
        # result=self.transpose_conv1d_2(result).transpose(1,2).contiguous().view(batch_size,max_seg_num*max_seg_length,-1)
        # print("deconv 2: ", result.shape)

        # batch_size * (max_seg_num * max_seg_length) * 50
        similar_1=self.similarity_linear1(h.transpose(1,2).contiguous().view(batch_size, max_seg_num*max_seg_length,-1))

        print("similar1: ", similar_1.shape)

        # batch_size * 50
        concept1_similar=self.similarity_linear2(concept1)
        concept2_similar=self.similarity_linear2(concept2)

        print("concept1_similar.unsqueeze(1): ", concept1_similar.unsqueeze(1).shape)

        # batch_size * (max_seg_num * max_seg_length) * 50
        # concept1_similar=concept1_similar.unsqueeze(1)*similar_1
        # concept2_similar=concept2_similar.unsqueeze(1)*similar_1

        # fusion unit -----new added
        mul1 = concept1_similar.unsqueeze(1)*similar_1
        mul2 = concept2_similar.unsqueeze(1)*similar_1

        add1 = concept1_similar.unsqueeze(1)+similar_1
        add2 = concept2_similar.unsqueeze(1)+similar_1

        op_fusion1 = torch.cat((concept1_similar.unsqueeze(1).expand(B,4000,300), similar_1, add1, mul1), dim=-1)
        op_fusion2 = torch.cat((concept2_similar.unsqueeze(1).expand(B,4000,300), similar_1, add2, mul2), dim=-1)

        # batch_size * (max_seg_num * max_seg_length) * 1
        concept1_score=self.MLP(op_fusion1)
        concept2_score=self.MLP(op_fusion2)

        # batch_size * (max_seg_num * max_seg_length) * 1
        concept1_score=torch.sigmoid(concept1_score)
        concept2_score=torch.sigmoid(concept2_score)

        # batch_size * max_seg_num * max_seg_length
        concept1_score=concept1_score.squeeze(-1).view(batch_size,max_seg_num,max_seg_length)
        concept2_score=concept2_score.squeeze(-1).view(batch_size,max_seg_num,max_seg_length)

        # batch_size * max_seg_num * max_seg_length
        return concept1_score,concept2_score,self_attention_wei, query_relevance_attention_wei, global_attention_wei


In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.


import networkx as nx
from sklearn.metrics import pairwise_distances
import numpy as np
import scipy.io

def process_video_mat(video_mat):
    result = []
    for shot_vec in video_mat:
        shot_vec= shot_vec[0][0]
        result.append(shot_vec)
    result = np.array(result)
    return result


def process_mat(mat):
    videos = mat['Tags'][0]
    result = []
    for video_mat in videos:
        video_mat = video_mat[0]
        video_data = process_video_mat(video_mat)
        result.append(video_data)
    return result


def load_videos_tag(mat_path="/content/drive/MyDrive/Research/data/evaluation/Tags.mat"):
    mat = scipy.io.loadmat(mat_path)
    videos_tag = process_mat(mat)
    return videos_tag

def semantic_iou(a, b):
    intersection = a * b
    intersection_num = sum(intersection)
    union = a + b
    union[union>0] = 1
    union_num = sum(union)
    if union_num != 0:
        return intersection_num / union_num
    else:
        return 0


def build_graph_from_pariwise_weights(weight_matrix):
    B = nx.Graph()
    bottom_nodes = list(map(lambda x: "b-{}".format(x), list(range(weight_matrix.shape[0]))))
    top_nodes = list(map(lambda x: "t-{}".format(x), list(range(weight_matrix.shape[1]))))
    edges = []
    for i in range(weight_matrix.shape[0]):
        for j in range(weight_matrix.shape[1]):
            weight = weight_matrix[i][j]
            edges.append(("b-{}".format(i), "t-{}".format(j), weight))
    B.add_weighted_edges_from(edges)
    return B


def calculate_semantic_matching(machine_summary, gt_summary, video_shots_tag, video_id):
    video_shots_tag = video_shots_tag[video_id]
    # print("v4 length: ",len(video_shots_tag))
    # print("v4 length: ",machine_summary)
    machine_summary_mat = video_shots_tag[machine_summary]
    gt_summary_mat = video_shots_tag[gt_summary]
    weights = pairwise_distances(machine_summary_mat, gt_summary_mat, metric=semantic_iou)
    B = build_graph_from_pariwise_weights(weights)
    matching_edges = nx.algorithms.matching.max_weight_matching(B)
    sum_weights = 0
    i = 0
    for edge in matching_edges:
        edge_data = B.get_edge_data(edge[0], edge[1])
        sum_weights += edge_data['weight']
        i += 1
    precision = sum_weights / machine_summary_mat.shape[0]
    recall = sum_weights / gt_summary_mat.shape[0]
    f1 = 2 * precision * recall / (precision + recall)
    return precision, recall, f1

In [ ]:
import os
import h5py
import torch
from torch.utils.data import DataLoader
# import matlab.engine

# from model import CHAN
# from dataset import UCTDataset
# from utils import load_pickle


class Runner():
    def __init__(self,config, train_vid, test_vid):
        self.config=config
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.train_vid = train_vid
        self.test_vid = test_vid
        self.loss_list = []
        self.f1_list = []
        self.train_f1_list = []
        os.environ["CUDA_VISIBLE_DEVICES"] = self.config["gpu"]
        self._build_dataloader()
        self._build_model()
        self._build_optimizer()
        self.max_f1=0
        self.max_p=0
        self.max_r=0


    def _build_model(self):
        self.model = CHAN(self.config).to(self.device)

    def _build_dataset(self):
        return UTEDataset(self.train_vid)

    def _build_dataloader(self):
        dataset=self._build_dataset()
        self.dataloader=DataLoader(dataset, batch_size=self.config["batch_size"], shuffle=True, num_workers=self.config["num_workers"])

    def _build_optimizer(self):
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.config["lr"], weight_decay=0.8)

    def output(self):
        print(" max_p = ",self.max_p," max_r = ",self.max_r," max_f1 = ",self.max_f1)


    def train(self):
        print("start to evaluate random result")
        # self.evaluate(self.test_vid,self.config["top_percent"])
        print("end to evaluate random result")

        criterion=torch.nn.BCELoss()
        self.model.train()
        for epoch in range(self.config["epoch"]):
            batch_count=0
            # loss_list = []
            for features,seg_len,concept1,concept2,concept1_GT,concept2_GT,mask_GT, q1t, q2t in self.dataloader:
                train_num=seg_len.shape[0]
                batch_count+=1
                self.optimizer.zero_grad()

                mask=torch.zeros(train_num,self.config["max_segment_num"],self.config["max_frame_num"],dtype=torch.bool).to(self.device)
                for i in range(train_num):
                    for j in range(len(seg_len[i])):
                        for k in range(seg_len[i][j]):
                            mask[i][j][k]=1

                # batch_size * max_seg_num * max_seg_length
                # print(f"Model: -----------")
                # print(f"Segment Length: {seg_len}")
                # print(f"Feature shape: {features.shape}")
                # print(f"Concept shape: {features.shape}")
                # print(f"Concept 1: {concept1}, Concept 2: {concept2}")
                # print(f"features: {features}")
                concept1_score,concept2_score, self_attention_wei, query_relevance_attention_wei, global_attention_wei=self.model(features.to(self.device),seg_len.to(self.device),concept1.to(self.device),concept2.to(self.device))
                # concept1_score,concept2_score, self_attention_wei, query_relevance_attention_wei, global_attention_wei=self.model(features,seg_len,concept1,concept2)

                loss=torch.zeros(1).to(self.device)
                for i in range(train_num):
                    concept1_score_tmp=concept1_score[i].masked_select(mask[i]).unsqueeze(0)
                    concept2_score_tmp=concept2_score[i].masked_select(mask[i]).unsqueeze(0)
                    concept1_GT_tmp=concept1_GT[i].masked_select(mask_GT[i]).unsqueeze(0)
                    concept2_GT_tmp=concept2_GT[i].masked_select(mask_GT[i]).unsqueeze(0)

                    # print(f"concept1_score: {concept1_score_tmp[:20]}, concept2_score: {concept2_score_tmp}")
                    # print(f"concept1_score_GT: {concept1_GT_tmp[:20]}, concept2_score_GT: {concept2_GT_tmp}")

                    loss1=criterion(concept1_score_tmp,concept1_GT_tmp.to(self.device))
                    loss2=criterion(concept2_score_tmp,concept2_GT_tmp.to(self.device))
                    # loss1=criterion(concept1_score_tmp,concept1_GT_tmp)
                    # loss2=criterion(concept2_score_tmp,concept2_GT_tmp)
                    # print(f"CE l1 = {loss1}")
                    # print(f"CE l2 = {loss2}")
                    loss+=loss1+loss2
                print("Loss: ", loss)
                # wandb.log({
                #   "train/loss": loss.item()/train_num
                # })

                if (batch_count+1)%5==0:
                    print("epoch ",epoch+1," batch ",batch_count+1," loss ",loss.item()/train_num)
                self.loss_list.append(loss.item()/train_num)
                loss.backward()
                self.optimizer.step()
            _, _, f1 = self.evaluate(self.test_vid,self.config["top_percent"]) # evaluating test dataset
            _, _, f1_train = self.evaluate(4,self.config["top_percent"]) # evaluating train dataset currently its 4
            # wandb.log({
            #   "epoch": epoch+1,
            #   "test/f1": f1,
            #   "train/f1": f1_train
            # })
            print("Loss: ",loss.item()/train_num)
            self.f1_list.append(f1)
            self.train_f1_list.append(f1_train)


    def evaluate(self,video_id,top_percent):
        """
        method takes test_video id and top_percent which is criteria for selecting a shot like 0.28 etc.
        """
        self.model.eval()
        f1_sum = 0
        p_sum = 0
        r_sum = 0

        # loading pickle file that is pretrained model to get features of query(text)
        embedding = load_pickle("/content/drive/MyDrive/Research/data/query_dictionary.pkl")

        evaluation_num = 0
        print("Evaluating on video:", video_id)
        # directory ../data/P01 contains Oracle summaries in txt file format
        # each concept1_concept2_oracle.txt file contains GT summary shot numbers
        for _, _, files in os.walk(f"/content/drive/MyDrive/Research/data/Oracle_summaries/P0{video_id}"): # getting all files in files
            evaluation_num = len(files)
            for file in files: # looping through each file or we can say training example that is concept1_concept2_oracle.txt
                # GTSummary list generation converting numbers given in file to list like [2, 3, ... ,443]
                summaries_GT = []
                with open(f"/content/drive/MyDrive/Research/data/Oracle_summaries/P0{video_id}/"+file, "r") as f:
                    for line in f.readlines():
                        summaries_GT.append(int(line.strip())) # this will append each GT shot number converting it to int and making its list like [1, 3,..434]
                    # print(summaries_GT)

                f = h5py.File(f'/content/drive/MyDrive/Research/data/processed/V{video_id}_resnet_avg.h5', 'r') # loading video features
                features=torch.tensor(f["features"][()], dtype=torch.float32)
                seg_len=torch.tensor(f["seg_len"][()], dtype=torch.int32)
                # features = torch.tensor(f["feature"][()], dtype=torch.float32) # converting feature values to tensor and loading features

                transfer={"Cupglass":"Glass","Musicalinstrument":"Instrument","Petsanimal":"Animal"}

                query1, query2 = file.split('_')[:2]
                if query1 in transfer:
                    query1=transfer[query1]
                if query2 in transfer:
                    query2=transfer[query2]

                query1 = torch.tensor(embedding[query1], dtype=torch.float32)
                query2 = torch.tensor(embedding[query2], dtype=torch.float32)
                # print(len(features))
                mask = torch.zeros(1, 20, 200, dtype=torch.bool).to(self.device)
                for i in range(1):
                    for j in range(len(seg_len.unsqueeze(0)[i])):
                        for k in range(seg_len.unsqueeze(0)[i][j]):
                            mask[i][j][k]=1
                q1_score, q2_score, self_attention_wei, query_relevance_attention_wei, global_attention_wei  = self.model(features.unsqueeze(0).to(self.device), seg_len.unsqueeze(0).to(self.device),query1.unsqueeze(0).to(self.device), query2.unsqueeze(0).to(self.device))
                # q1_score, q2_score, self_attention_wei, query_relevance_attention_wei, global_attention_wei = self.model(features.unsqueeze(0), seg_len.unsqueeze(0),query1.unsqueeze(0), query2.unsqueeze(0))
                # print("q1:", q1_score.shape)
                # q1_score dims [1, 2783, 1]

                score = q1_score + q2_score
                # print(score.shape)  # dims [1, 2783, 1]
                # score = score.transpose(1,2)
                score = score.masked_select(mask) # dims [7745089]
                # print(score.shape)

                _,top_index=score.topk(int(score.shape[0]*0.02))

                if 3588 in top_index and video_id == 4:
                    print("removing 3588")
                    mk = top_index != 3588
                    top_index = top_index[mk]


                # print("before summaries_GT.shape : ", summaries_GT)
                if 3692 in summaries_GT and video_id == 2:
                    # removing this index from video 2 using remove() method because summaries_GT is python list and not the tensor
                    print("removing 3692")
                    summaries_GT.remove(3692)
                    # mk = top_index != 3692
                    # print(mk)
                    # top_index = top_index[mk]

                    # mk = summaries_GT != 3692
                    # summaries_GT = summaries_GT[mk]


                # print("before summaries_GT.shape : ", summaries_GT)
                top_index_to_print = top_index + 1
                print(file.split('_')[:])
                print("evaluation video: ", video_id-1)
                # print(top_index)
                print(top_index_to_print)
                video_shots_tag = load_videos_tag(mat_path="/content/drive/MyDrive/Research/data/evaluation/Tags.mat")
                p, r, f1 = calculate_semantic_matching(list(top_index.cpu().numpy()), summaries_GT, video_shots_tag, video_id=video_id-1) # video id = 0,1,2,3

                f1_sum += f1
                p_sum += p
                r_sum += r
                # print(p, r , f1)
                print("p ", p, "r ", r, "f1 ", f1)
                # self.dataset.append(file[:file.find("_oracle.txt")]+"_"+"1")            print("p ", p_sum/evaluation_num, "r ", r_sum/evaluation_num, "f1 ", f1_sum/evaluation_num)
            print("p ", p_sum/evaluation_num, "r ", r_sum/evaluation_num, "f1 ", f1_sum/evaluation_num)
            self.model.train()
            return (p_sum/evaluation_num), (r_sum/evaluation_num), (f1_sum/evaluation_num)



In [ ]:
exp1 = Runner(config, [2,3,4], 1)

In [ ]:
exp1.train()

start to evaluate random result
end to evaluate random result
conv1 torch.Size([100, 2048, 100])
conv2 torch.Size([100, 2048, 50])
conv3 torch.Size([100, 2048, 25])
conv4 torch.Size([100, 2048, 19])
conv5 torch.Size([100, 2048, 10])
conv6 torch.Size([100, 4096, 10])
conv7 torch.Size([100, 4096, 10])
conv8 torch.Size([100, 256, 10])
k shape : torch.Size([100, 10, 256]) | q shape: torch.Size([100, 10, 256])
wei shape:  torch.Size([100, 10, 10])
k shape : torch.Size([100, 10, 256]) | q shape: torch.Size([100, 10, 256])
wei shape:  torch.Size([100, 10, 10])
k shape : torch.Size([10, 100, 256]) | q shape: torch.Size([10, 100, 256])
wei shape:  torch.Size([10, 100, 100])
deconv1 torch.Size([100, 1024, 19])
deconv2 torch.Size([100, 1024, 200])
similar1:  torch.Size([5, 4000, 300])
concept1_similar.unsqueeze(1):  torch.Size([5, 1, 300])
Loss:  tensor([6.9563], device='cuda:0', grad_fn=<AddBackward0>)
conv1 torch.Size([100, 2048, 100])
conv2 torch.Size([100, 2048, 50])
conv3 torch.Size([100, 20

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 6.25 GiB. GPU  (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:924.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


conv1 torch.Size([100, 2048, 100])
conv2 torch.Size([100, 2048, 50])
conv3 torch.Size([100, 2048, 25])
conv4 torch.Size([100, 2048, 19])
conv5 torch.Size([100, 2048, 10])
conv6 torch.Size([100, 4096, 10])
conv7 torch.Size([100, 4096, 10])
conv8 torch.Size([100, 256, 10])
k shape : torch.Size([100, 10, 256]) | q shape: torch.Size([100, 10, 256])
wei shape:  torch.Size([100, 10, 10])
k shape : torch.Size([100, 10, 256]) | q shape: torch.Size([100, 10, 256])
wei shape:  torch.Size([100, 10, 10])
k shape : torch.Size([10, 100, 256]) | q shape: torch.Size([10, 100, 256])
wei shape:  torch.Size([10, 100, 100])
deconv1 torch.Size([100, 1024, 19])
deconv2 torch.Size([100, 1024, 200])
similar1:  torch.Size([5, 4000, 300])
concept1_similar.unsqueeze(1):  torch.Size([5, 1, 300])
Loss:  tensor([6.8377], device='cuda:0', grad_fn=<AddBackward0>)
conv1 torch.Size([100, 2048, 100])
conv2 torch.Size([100, 2048, 50])
conv3 torch.Size([100, 2048, 25])
conv4 torch.Size([100, 2048, 19])
conv5 torch.Size([1

KeyboardInterrupt: 

In [ ]:
wandb.finish()

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test/f1,▆▇▁▄▄▃▁▆█▇▇▇█▅█▇███▆
train/f1,██▆▆▄▁▁▆██████▇▇█▅██
train/loss,█▇▆▄▅▃▇▄▄▅▃▅▃▄▃▅▅▂▃▄▄▄▂▄▄▂▄▃▄▄▄▃▃▂▂▄▅▁▃▁
epoch,20
test/f1,0.45153
train/f1,0.36427
train/loss,0.86613


In [ ]:
model_scripted = torch.jit.script(exp1.model) # Export to TorchScript
model_scripted.save('/content/drive/MyDrive/Research/FCSN_ATTN_FU/train4_test1.pt')

In [ ]:
# del exp1